In [1]:
"""
Convert the family screening data to its most interpretable form

TODO: Move this to a more extensible format
"""

import sys
import importlib
import pandas as pd
import numpy as np
sys.path.append("C:\\Users\\Ola\\Documents\\Research\\Implementations\\EvaluatingXAI\\explanation-toolkit")

In [3]:
from examples.family_screening import family_screening_helpers as helpers
importlib.reload(helpers)

features = helpers.get_features()
feature_types = helpers.get_feature_types(features)

X, y = helpers.load_data(features)

feature_mappings, defaults = helpers.load_feature_mappings()

C:\Users\Ola\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: DtypeWarning: Columns (222) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
feature_mappings.head()
defaults.head()

,Feature,Value
0,PRI_FOCUS_GENDER,Male
1,PRI_CBMS_FOCUS_CD,Never
2,PRI_CBMS_FOCUS_CH,Never
3,PRI_CBMS_FOCUS_CP,Never
4,PRI_CBMS_FOCUS_CW,Never


In [5]:
# Get the list of features and default values
features = X.columns
one_hot_features = feature_mappings["Feature"]
compound_features = feature_mappings["InterpretableFeature"].unique()

features = features.drop(labels=one_hot_features).append(pd.Index(compound_features))
default_dataframe = pd.DataFrame(index=X.index, columns=features, dtype=object).replace(np.nan, 'None')

In [6]:
to_append = pd.concat([defaults["Value"]] * default_dataframe.shape[0], axis=1, ignore_index=True).transpose()
default_dataframe[defaults["Feature"]] = to_append


In [23]:
# Convert instances

for i in range(X.shape[0]):
    x = X.iloc[i]
    for item in x.items():
        mapping = feature_mappings.loc[feature_mappings['Feature'] == item[0]]
        if mapping.size == 0:
            default_dataframe[i, item[0]] = item[1]
        else:
            if(item[1] > 0):
                default_dataframe.iloc[i][mapping["InterpretableFeature"]] = mapping["Value"]


C:\Users\Ola\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Ola\Anaconda3\lib\site-packages\pandas\core\series.py:1085: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)


MemoryError: 